In [ ]:
%pip install ragas

In [ ]:
%pip uninstall -y sentence-transformers

In [ ]:
%pip install InstructorEmbedding
%pip install sentencepiece

In [ ]:
%pip install sentence-transformers
%pip install protobuf
%pip install transformers accelerate


In [ ]:
import ragas
ragas.__version__

In [ ]:
import pandas as pd
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, context_precision, context_recall, faithfulness
from ragas.embeddings.base import Embeddings
from typing import List
from sentence_transformers import SentenceTransformer
from langchain_community.llms import Ollama


In [ ]:
class RagasEmbeddingWrapper(Embeddings):
    def __init__(self, model_name_or_path: str):
        self.model_name_or_path = model_name_or_path

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        # buat instance setiap kali embed untuk menghindari serialisasi instance
        model = SentenceTransformer(self.model_name_or_path, device="cuda")
        return model.encode(texts).tolist()

    def embed_query(self, text: str) -> List[float]:
        return self.embed_documents([text])[0]

In [ ]:
emb_wrapper = RagasEmbeddingWrapper(
    r"C:/Users/PIPP/Documents/TutorDek/TutorCerdas/server/PretrainedModel/multilingual-e5-large"
)

In [ ]:
llm = Ollama(
    model="hf.co/ojisetyawan/llama3-8b-cpt-sahabatai-v1-instruct-Q4_K_M-GGUF",
    temperature=0.2
)

In [ ]:
df = pd.read_csv("C:/Users/PIPP/Documents/TutorDek/TutorCerdas/server/data_evaluasi/Tutor Cerdas pada LMS (Responses) - Copy of PTKI.csv")
df.head()

In [ ]:
def combine_context(row):
    ctxs = []
    for i in range(1, 5):
        col = f"context {i}"
        if isinstance(row[col], str) and row[col].strip() != "":
            ctxs.append(row[col])
    return ctxs

df["contexts"] = df.apply(combine_context, axis=1)
# df2["contexts"] = df2.apply(combine_context, axis=1)
# df3["contexts"] = df2.apply(combine_context, axis=1)

In [ ]:
dataset = Dataset.from_pandas(df)
dataset = dataset.rename_columns({
    "user_input": "question",
    "response": "answer"
})

In [ ]:
results = evaluate(
    dataset=dataset,
    metrics=[answer_relevancy, context_precision, context_recall, faithfulness],
    llm=llm,
    embeddings=emb_wrapper,
    batch_size=1,
    show_progress=True
)

In [ ]:
result_df = results.to_pandas()
result_df = result_df.fillna(0)
# result_df = pd.concat([result_df1, result_df2, result_df3], ignore_index=True)
ragas_columns = ["answer_relevancy", "context_precision", "context_recall", "faithfulness"]
result_df["ragas_score"] = result_df[ragas_columns].mean(axis=1, skipna=True)
result_df

In [ ]:
overall_ragas = float(result_df["ragas_score"].mean())
print(f"Skor Evaluasi RAG Keseluruhan : {overall_ragas:.10f}")
print(f"Presentase Skor Evaluasi RAG : {overall_ragas*100:.2f}%")

In [ ]:
result_df.to_excel("EvaluasiPTKI.xlsx", index=False)